In [1]:
import tensorflow as tf
from tensorflow.python.ops import rnn,rnn_cell
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)

def weight_variable(shape, name):
    return tf.Variable(tf.truncated_normal(shape = shape, stddev = 0.1), name)

def bias_variable(shape, name):
    return tf.Variable(tf.zeros(shape = shape), name)

def plot_n_reconstruct(origin_img, reconstruct_img, n = 10):

    plt.figure(figsize=(2 * 10, 4))

    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(origin_img[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstruct_img[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
n_input=28
n_steps=28
n_hidden=128
n_classes=10

x=tf.placeholder("float",[None,n_steps,n_input])
y=tf.placeholder("float",[None,n_classes])

weights={
    "w_fc":weight_variable([n_hidden,n_classes],"w_fc")
}
biases={
    "b_fc":bias_variable([n_classes],"b_fv")
}

In [3]:
x_transpose = tf.transpose(x, [1, 0, 2])
print("x_transpose shape: %s" % x_transpose.get_shape())

x_reshape = tf.reshape(x_transpose, [-1, n_input])
print("x_reshape shape: %s" % x_reshape.get_shape())

#order change in tf.split
x_split = tf.split(x_reshape, n_steps, 0)
print("type of x_split: %s" % type(x_split))
print("length of x_split: %d" % len(x_split))
print("shape of x_split[0]: %s" % x_split[0].get_shape())

x_transpose shape: (28, ?, 28)
x_reshape shape: (?, 28)
type of x_split: <type 'list'>
length of x_split: 28
shape of x_split[0]: (?, 28)


In [5]:
basic_rnn_cell=tf.contrib.rnn.BasicRNNCell(n_hidden)
h,states=tf.contrib.rnn.static_rnn(basic_rnn_cell,x_split,dtype=tf.float32)


In [6]:
#fully connected layer

h_fc=tf.matmul(h[-1],weights['w_fc']+biases['b_fc'])
y_=h_fc

#cost function
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=h_fc,labels=y))
optimizer=tf.train.AdamOptimizer(0.01).minimize(cost)

#accuracy function
correct_prediction=tf.equal(tf.argmax(y_,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [7]:
batch_size=100
init_op=tf.global_variables_initializer()
sess=tf.InteractiveSession()
sess.run(init_op)

variables_names=[v.name for v in tf.trainable_variables()]

for step in range(5000):
    batch_x,batch_y=mnist.train.next_batch(batch_size)
    batch_x=np.reshape(batch_x,(batch_size,n_steps,n_input))
    cost_train,accuracy_train,states_train,rnn_out=sess.run([cost,accuracy,states,h[-1]],feed_dict={x:batch_x,y:batch_y})
    values=sess.run(variables_names)
    rnn_out_mean=np.mean(rnn_out)
    
    for k,v in zip(variables_names,values):
        if k== 'rnn/basic_rnn_cell/weights:0': #k=='RNN/BasicRNNCell/Linear/Matrix:0':
            w_rnn_mean=np.mean(v)
    if step<1500:
        if step % 100 == 0:
            print("step %d, loss %.5f, accuracy %.3f, mean of rnn weight %.5f, mean of rnn out %.5f" % (step, cost_train, accuracy_train, w_rnn_mean, rnn_out_mean))
    else:
        if step%1000 == 0: 
            print("step %d, loss %.5f, accuracy %.3f, mean of rnn weight %.5f, mean of rnn out %.5f" % (step, cost_train, accuracy_train, w_rnn_mean, rnn_out_mean))
            
    optimizer.run(feed_dict={x:batch_x,y:batch_y})
    
print("training session done")

step 0, loss 2.32661, accuracy 0.100, mean of rnn weight 0.00007, mean of rnn out -0.01312
step 100, loss 2.38740, accuracy 0.110, mean of rnn weight 0.00041, mean of rnn out -0.00749
step 200, loss 1.97791, accuracy 0.220, mean of rnn weight 0.00027, mean of rnn out -0.01693
step 300, loss 2.53584, accuracy 0.150, mean of rnn weight 0.00156, mean of rnn out 0.03125
step 400, loss 2.31662, accuracy 0.140, mean of rnn weight 0.00271, mean of rnn out 0.04397
step 500, loss 2.33348, accuracy 0.070, mean of rnn weight 0.00271, mean of rnn out 0.04551
step 600, loss 2.31514, accuracy 0.130, mean of rnn weight 0.00272, mean of rnn out 0.03230
step 700, loss 2.37768, accuracy 0.110, mean of rnn weight 0.00273, mean of rnn out 0.03187
step 800, loss 2.33249, accuracy 0.120, mean of rnn weight 0.00275, mean of rnn out 0.03247
step 900, loss 2.28118, accuracy 0.150, mean of rnn weight 0.00277, mean of rnn out 0.03504
step 1000, loss 2.39003, accuracy 0.100, mean of rnn weight 0.00280, mean of rn

In [8]:
cost_test, accuracy_test = sess.run([cost, accuracy], feed_dict={x: np.reshape(mnist.test.images, [-1, 28, 28]), y: mnist.test.labels})
print("final loss %.5f, accuracy %.5f" % (cost_test, accuracy_test) )

final loss 2.37063, accuracy 0.09820


In [9]:
print h[-1].eval(feed_dict={x: np.reshape(mnist.test.images, [-1, 28, 28]), y: mnist.test.labels})[0,:]

[ 1.         -0.99998403  1.          1.          0.99994344 -0.99999428
 -1.          1.         -0.99999982  1.          1.          1.
 -0.99999988 -0.9999637  -1.         -0.99999183  0.99999082 -1.
  0.99999988  1.          1.         -0.99999994 -0.9999997  -0.99999988
 -1.          1.          1.          1.          1.          0.99999928
  0.99999642  1.         -1.          1.         -1.          1.         -1.
  0.99996853  1.         -1.          1.         -1.          1.         -1.
 -0.99999928 -0.99999994  0.99999982  1.          0.99999917  1.         -0.9999882
 -1.          1.          0.99999857  1.         -0.99999994  1.00000012
 -0.9999975  -1.          1.         -1.         -1.         -1.          1.
  0.99999797 -0.99999994 -1.         -1.          1.         -1.         -1.
  1.         -0.9999975  -1.         -1.          1.         -0.99994415
 -0.99999994  1.         -1.          1.          1.          1.         -1.
 -0.99999541 -1.         -1.        

In [12]:
#LSTM
tf.reset_default_graph()

x=tf.placeholder("float",[None,n_steps,n_input])
y=tf.placeholder("float",[None,n_classes])

weights={
    "w_fc":weight_variable([n_hidden,n_classes],"w_fc")
}
biases={
    "b_fc":bias_variable([n_classes],"b_fv")
}
x_transpose = tf.transpose(x, [1, 0, 2])
x_reshape = tf.reshape(x_transpose, [-1, n_input])
x_split = tf.split(x_reshape, n_steps, 0)
lstm_cell=tf.contrib.rnn.BasicLSTMCell(n_hidden,forget_bias=1.0)
h,states=tf.contrib.rnn.static_rnn(lstm_cell,x_split,dtype=tf.float32)

h_fc=tf.matmul(h[-1],weights['w_fc']+biases['b_fc'])
y_=h_fc

#cost function
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=h_fc,labels=y))
optimizer=tf.train.AdamOptimizer(0.01).minimize(cost)

#accuracy function
correct_prediction=tf.equal(tf.argmax(y_,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

batch_size=100
init_op=tf.global_variables_initializer()
sess=tf.InteractiveSession()
sess.run(init_op)

variables_names=[v.name for v in tf.trainable_variables()]

for step in range(5000):
    batch_x,batch_y=mnist.train.next_batch(batch_size)
    batch_x=np.reshape(batch_x,(batch_size,n_steps,n_input))
    cost_train,accuracy_train,states_train,rnn_out=sess.run([cost,accuracy,states,h[-1]],feed_dict={x:batch_x,y:batch_y})
    values=sess.run(variables_names)
    rnn_out_mean=np.mean(rnn_out)
    
    for k,v in zip(variables_names,values):
        if k== 'rnn/basic_lstm_cell/weights:0':
            w_rnn_mean=np.mean(v)
    if step<1500:
        if step % 100 == 0:
            print("step %d, loss %.5f, accuracy %.3f, mean of rnn weight %.5f, mean of rnn out %.5f" % (step, cost_train, accuracy_train, w_rnn_mean, rnn_out_mean))
    else:
        if step%1000 == 0: 
            print("step %d, loss %.5f, accuracy %.3f, mean of rnn weight %.5f, mean of rnn out %.5f" % (step, cost_train, accuracy_train, w_rnn_mean, rnn_out_mean))
            
    optimizer.run(feed_dict={x:batch_x,y:batch_y})
    
print("training session done")

step 0, loss 2.30438, accuracy 0.110, mean of rnn weight 0.00019, mean of rnn out 0.00221
step 100, loss 0.47128, accuracy 0.890, mean of rnn weight -0.00620, mean of rnn out -0.01431
step 200, loss 0.14613, accuracy 0.960, mean of rnn weight -0.00662, mean of rnn out -0.01598
step 300, loss 0.32769, accuracy 0.930, mean of rnn weight -0.00778, mean of rnn out -0.00711
step 400, loss 0.26119, accuracy 0.930, mean of rnn weight -0.00861, mean of rnn out 0.00630
step 500, loss 0.02981, accuracy 0.990, mean of rnn weight -0.00875, mean of rnn out 0.01064
step 600, loss 0.16207, accuracy 0.960, mean of rnn weight -0.00971, mean of rnn out 0.00136
step 700, loss 0.06180, accuracy 0.980, mean of rnn weight -0.00948, mean of rnn out -0.00026
step 800, loss 0.15753, accuracy 0.960, mean of rnn weight -0.00958, mean of rnn out 0.00031
step 900, loss 0.02525, accuracy 1.000, mean of rnn weight -0.00992, mean of rnn out 0.00317
step 1000, loss 0.05837, accuracy 0.980, mean of rnn weight -0.01085,

In [13]:
cost_test, accuracy_test = sess.run([cost, accuracy], feed_dict={x: np.reshape(mnist.test.images, [-1, 28, 28]), y: mnist.test.labels})
print("final loss %.5f, accuracy %.5f" % (cost_test, accuracy_test) )

final loss 0.06165, accuracy 0.98360
